# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **

A classificação é utilizada quando devemos mapear uma entrada para um rótulo discreto, algo como Verdadeiro ou Falso.
A regressão é utilizada quando queremos prever o valor de um determinado ponto em um espectro contínuo de valores.

Um exemplo para indicar isso é: 
 - Classificação: Uma imagem é de um homem ou de uma mulher?;
 - Regressão: Prever valor de um imóvel baseado em sua localização;
 
Com essas informações, podemos analisar que o objetivo desse projeto é: Verificar alunos que precisam de intervenção e alunos que não precisam de intervenção. Como essa é uma saída discreta, é considerado um problema de classificação.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [22]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [23]:
# TODO: Calcule o número de estudante
n_students = student_data['school'].count()

# TODO: Calcule o número de atributos
n_features = len(student_data.columns) - 1

# TODO: Calcule o número de alunos aprovados
n_passed = student_data[student_data['passed']=='yes']['passed'].count()

# TODO: Calcule o número de alunos reprovados
n_failed = student_data[student_data['passed']=='no']['passed'].count()

# TODO: Calcule a taxa de graduação
grad_rate = (np.float32(n_passed) / np.float32(n_students)) * 100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [24]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [25]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [26]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui

# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
from sklearn.model_selection import ShuffleSplit
rs = ShuffleSplit(n_splits=1, train_size=num_train, test_size=num_test, random_state=50)

for train_index, test_index in rs.split(X_all):
    X_train = X_all.iloc[train_index]
    y_train = y_all.iloc[train_index]
    X_test = X_all.iloc[test_index]
    y_test = y_all.iloc[test_index]

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **

De acordo com as pesquisas realizadas, os algoritmos que mais atendem a realidade do problema são: Máquinas de vetores de suporte, Naive Bayes e Árvores de decisão;

As **SVM** (Máquina de vetores de suporte) é utilizada em algoritmos para avaliar se pessoas possuem ou não possuem determinada doença, reconhecimento de escrita a mão, categorização de textos, predição de preços de ações etc. As vantagens desse tipo de algoritmo é que dá algumas garantias em relação a overfitting, trabalha bem mesmo que os dados no training set não sejam linearmente separáveis. Porém, tem como desvantagens que consome muita memória, é complicado escolher o melhor kernel e não é escalável para datasets muito grandes. Esse algoritmo se mostra interessante, pois o dataset não é tão grande e esse algoritmo consegue ter uma precisão alta mesmo nesse caso.

O **Naive Bayes** é utilizado em programas para análise de sentimentos, classificação de texto (spam não spam), sistemas de recomendação e reconhecimento de faces. As vantagens é que esse algoritmo é muito simples, não precisa de muitos dados para treinamento e tem uma performance muito boa. A desvantagem é que por ser muito simples, para determinados casos ele pode ser facilmente substituído por algum algotitmo específico para o problema, que pode ser mais configurável e customizável. Foi escolhido para análise por ser muito simples e que pode consumir menos recursos para fazer as predições com uma acuracidade interessante.

As **árvores de decisão** são utilizadas para decisões sobre investimentos, decisão sobre construção ou compra, decisão sobre compra de empresas, análise de inadimplentes etc. As vantagens é que são fáceis de interpretar e explicar, são não parametrizáveis, dessa forma você não precisa se preocupar com dados isolados. A desvantagen é que a chance de overfitting é alta e o consumo é razoavelmente alto dependendo da complexidade do seu modelo. Foi escolhido esse modelo pois a acuracidade dele é alta, e é possível evitar o overfitting limitando o número de subdivisões.

Referências:
 - https://docs.microsoft.com/en-us/azure/machine-learning/studio/algorithm-choice
 - https://www.datasciencecentral.com/profiles/blogs/want-to-know-how-to-choose-machine-learning-algorithm
 - https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms
 - https://elitedatascience.com/machine-learning-algorithms

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [27]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [28]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

# TODO: Inicialize os três modelos
clf_A = svm.SVC(random_state=50)
clf_B = GaussianNB()
clf_C = tree.DecisionTreeClassifier(random_state=50)

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print "\n"
print "\n"
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print "\n"
print "\n"
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "\n"
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Treinando um SVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8707.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7681.


Treinando um SVC com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0030 segundos
As previsões foram feitas em 0.0030 segundos.
Pontuação F1 para o conjunto de treino: 0.8738.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7943.


Treinando um SVC com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0060 segundos
As previsões foram feitas em 0.0050 segundos.
Pontuação F1 para o conjunto de treino: 0.8595.
As previsões foram feitas em 0.0020 segundos.
Pontuação F1 para o conjunto de teste: 0.7832.




Treinando um GaussianNB com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0000 segundos
As previsões foram feitas em 0.0000 segundos.
Pontua

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - Máquinas de vetores de suporte**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |    0.0020s           |             0.0020s         |       0.8707               |        0.7681        |
| 200                                |    0.0030s           |             0.0030s         |       0.8738               |        0.7943        |
| 300                                |    0.0060s           |             0.0050s         |       0.8738               |        0.7832        |

** Classificador 2 - Naive Bayes**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |       0.0010s        |     0.0000s                 |     0.8305                 |    0.6721            |
| 200                                |       0.0000s        |     0.0010s                 |     0.8078                 |    0.6957            |
| 300                                |       0.0010s        |     0.0010s                 |     0.7931                 |    0.6724            |

** Classificador 3 - Árvore de decisão**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |     0.0000s          |     0.0010s                 |      1.0000                |     0.7692           |
| 200                                |     0.0010s          |     0.0000s                 |      1.0000                |     0.6723           |
| 300                                |     0.0010s          |     0.0010s                 |      1.0000                |     0.6935           |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **

Com base na pontuação F1 para os 3 modelos e os 3 treinamentos, podemos observar que a SVM possui a melhor média de score no conjunto de teste. 
A árvore de decisão se mostrou muito boa no score F1 do conjunto de treinamento porém uma média baixa no score do conjunto de teste, o que pode indicar um overfitting, logo, descartamos esse modelo.
O Naive bayes se mostrou constante nos três treinamentos, porém com uma média muito baixa de score no conjunto de teste, descartaremos esse modelo também.

Dessa forma, utilizaremos o SVM para resolução desse problema.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **

De acordo com os testes de performance realizados, o modelo utilizado será o SVM (em português, Máquinas de vetores de suporte), pois esse modelo se mostrou o mais assertivo em relação aos outros modelos estudados para o problema em questão.

Para explicar o funcionamento dessa técnica vamos nomear os vetores de suporte como pontos, de acordo com a imagem abaixo
![SVM-img1](http://ap.imagensbrasil.org/images/2018/02/19/img1.png)

O objetivo dessa técnica é traçar uma linha que separe esses pontos, porém para isso adicionaremos uma nova nomenclatura para a explicação que é o conceito de margem, esse conceito diz que a linha traçada entre esses pontos deve maximizar a distância dos pontos para essa linha, ou seja, essa linha não pode passar mais perto de um tipo de ponto do que o outro, a imagem abaixa ilustra isso mais facilmente
![SVM-img2](http://ap.imagensbrasil.org/images/2018/02/19/img2.png)

Podemos ver que na imagem acima, o primeiro gráfico não tem uma margem muito boa dos pontos vermelhos, pois está mais próximo do ponto vermelho do que do ponto azul, o terceiro gráfico tem o mesmo conceito porém se aproxima muito dos pontos azuis. O gráfico do meio parece o mais correto pois a linha traçada separa os pontos com o máximo de margem possível entre os pontos (lembrando que as ilustrações são apenas para exemplificar o funcionamento, não estão com dados reais e exatos).

Dessa forma, a SVM aprende com os dados dos alunos passados a melhor linha que separará os alunos que precisam de intervenção para os que não precisam.
Porém o mais legal do porquê esse algoritmo funciona muito bem no nosso caso, é que mesmo que alguns alunos que tenham indicadores bons, e precisem de intervenção, ou alunos que tem indicadores ruins e não precisam de intervenção, o que deixaria nosso gráfico similar a forma abaixo:
![SVM-img3](http://ap.imagensbrasil.org/images/2018/02/19/img3.png)

Como podemos ver na imagem acima, não é possível traçar uma linha reta que separe os pontos vermelhos dos pontos azuis. Para isso explicaremos mais um conceito da SVM, chamado de 'Truque de Kernel', esse truque permite que esses pontos sejam separados corretamente. Parece impossível? mas não é! Ficará mais fácil caso esses pontos sejam imaginados como bolas de gude em cima de uma mesa. O truque do kernel é que iremos levantar as bolas azuis e deixar as bolas vermelhas na mesa, e colocaremos uma separação entre elas, por exemplo, iremos levantar apenas as bolas azuis, colocar uma folha de papel acima das bolas vermelhas e depois colocamos novamente as bolas azuis, dessa forma, elas estarão separadas pela folha de papel. Como mostra a imagem abaixo:
![SVM-img3](http://i.imgur.com/WuxyO.png)

Podemos confiar na técnica utilizada pois mesmo nesses casos difíceis, ela funciona muito bem, graças ao truque do kernel, que transforma nossa entrada em uma espécie de entrada tridimensional e torna possível a separação de pontos que não são linearmente separáveis.

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [30]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.svm import SVC

#Referências: https://github.com/darkdreamingdan/mlnd-student-intervention/blob/master/student_intervention.ipynb
    
# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
     'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
     'kernel': ['rbf', 'sigmoid', 'poly']
     }

# TODO: Inicialize o classificador
clf = SVC()

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf,param_grid = parameters, scoring = f1_scorer)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

As previsões foram feitas em 0.0050 segundos.
O modelo calibrado tem F1 de 0.9742 no conjunto de treinamento.
As previsões foram feitas em 0.0020 segundos.
O modelo calibrado tem F1 de 0.7755 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **

O score F1 do conjunto de treinamento no modelo calibrado é 0.9742 enquanto no modelo padrão é de 0.8738, o que mostra um aumento de cerca de 11,5%, enquanto que no conjunto de testes o score foi de 0.7832 para 0.7755 o que mostra uma perda de cerca de 1%.

No geral o ganho é maior do que a perda, o que indica que valeu a pena realizar a calibração do modelo.

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.